In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import os

options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Run in headless mode
download_directory = os.path.abspath(os.path.join(os.getcwd(), "inputs"))
prefs = {"download.default_directory": download_directory}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# List of URLs to download: https://www.pimco.com/us/en/product-finder?filters=assetClass%3Dfxi%26fixedIncome%3Dinf_ass 
urls = [
    "https://fund-ui.pimco.com/fund-detail-api/api/funds/documents/content/view?documentId=Gz5zDyrmed3KGHFXm6XTTydbOXIsX84jJlyfQMH7fO0",
    "https://fund-ui.pimco.com/fund-detail-api/api/funds/documents/content/view?documentId=1sv1xNFcBSE2xYkLCvV1ICdbOXIsX84jJlyfQMH7fO0",
    "https://fund-ui.pimco.com/fund-detail-api/api/funds/documents/content/view?documentId=wYHetYuoNap8QwQWXKZYYidbOXIsX84jJlyfQMH7fO0",
    "https://fund-ui.pimco.com/fund-detail-api/api/funds/documents/content/view?documentId=k3j1YqIgD4q2honL01OpWydbOXIsX84jJlyfQMH7fO0",
    "https://fund-ui.pimco.com/fund-detail-api/api/funds/documents/content/view?documentId=w2sb9Hpa4VOR6cNSV9E9uidbOXIsX84jJlyfQMH7fO0"
]

# Loop through each URL and download the file
for url in urls:
    before_download = set(os.listdir(download_directory))
    driver.get(url)
    time.sleep(3)  # Wait for the download to complete
driver.quit()

print("All downloads completed!")

input_directory = download_directory  
output_filename = 'PIMCO_fund_analysis_output.xlsx'

df_dict = {}

for filename in os.listdir(input_directory):
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        filepath = os.path.join(input_directory, filename)
        
    
        df = pd.read_excel(filepath, header=[15, 16])
        df.columns = [' '.join(col).strip() for col in df.columns.values]
        df.rename({'Unnamed: 0_level_0 Unnamed: 0_level_1':'INSTRUMENT'}, axis=1, inplace=True)
        
        # Filter and clean the dataframe
        df = df[['INSTRUMENT','RATE','MATURITY DATE','AMOUNT OWNED','% OF NET ASSETS']]
        df = df[df['INSTRUMENT'] == 'U.S. Treasury Inflation Protected Securities']
        df['RATE'] = (df['RATE'] * 100).astype(float).round(3)
        df['AMOUNT OWNED (MM)'] = (df['AMOUNT OWNED'] / 1000000).astype(float).round(2)
        df['MATURITY DATE'] = pd.to_datetime(df['MATURITY DATE']).dt.date
        df = df[['RATE','MATURITY DATE','AMOUNT OWNED (MM)']].groupby(['RATE','MATURITY DATE'], as_index=False)['AMOUNT OWNED (MM)'].sum()
        df = df.sort_values(by='MATURITY DATE').reset_index(drop=True)
   
        fund_name = os.path.splitext(os.path.basename(filename))[0]
        df_dict[fund_name] = df

total_df = pd.concat(df_dict.values(), ignore_index=True).groupby(['RATE','MATURITY DATE'], as_index=False)['AMOUNT OWNED (MM)'].sum()
total_df = total_df.sort_values(by='MATURITY DATE').reset_index(drop=True)
df_dict['Total_Aggregated'] = total_df

with pd.ExcelWriter(output_filename) as writer:
    for fund_name, df in df_dict.items():
        df.to_excel(writer, sheet_name=fund_name.replace('PIMCO Funds ',"")[:30], index=False)

print(f"Data has been written to {output_filename}")

All downloads completed!
Data has been written to PIMCO_fund_analysis_output.xlsx
